In [1]:
from sklearn.datasets import make_classification
X,y = make_classification(n_samples =1000, n_features=5, n_informative=2,n_redundant=1,random_state=1)
print(X.shape, y.shape)

(1000, 5) (1000,)


# Optimizer Hyperparameter Perceptron
+ eta : Learning rate
+ alpha : regularization
+ RepeatedStratifiedKFold là hàm chia ra làm k fold để đánh giá chéo với tham số n_split là số k fold, n_repeat là số lần lặp lại quá trình trên

In [2]:
import numpy as np
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.linear_model import Perceptron

In [3]:
model = Perceptron()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model,X,y,scoring='accuracy', cv=cv, n_jobs=-1)
print("Scores = ",np.mean(scores))
print("Std = ",np.std(scores))


Scores =  0.7863333333333334
Std =  0.06887105503927045


In [4]:
def objective(X,y, cfg):
  eta, alpha = cfg
  model = Perceptron(penalty='elasticnet', alpha=alpha,eta0=eta)
  cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
  scores = cross_val_score(model,X,y,scoring='accuracy', cv=cv, n_jobs=-1)
  result = np.mean(scores)
  return result

def step(cfg, step_size):
  eta, alpha = cfg
  new_eta = eta + np.random.rand() * step_size
  if new_eta <= 0.0:
    new_eta = 1e-8

  new_alpha = alpha + np.random.rand() * step_size
  if new_alpha < 0.0:
    new_alpha = 0.0
  return [new_eta, new_alpha]

def hillclimbing(X,y, objective, n_iter, step_size):

  solution = [np.random.rand(), np.random.rand()]
  solution_eval = objective(X,y, solution)

  for i in range(n_iter):
    candidate = step(solution, step_size)
    candidate_eval = objective(X,y, candidate)

    if candidate_eval >= solution_eval:
      solution, solution_eval = candidate, candidate_eval

  return [solution, solution_eval]


In [5]:
n_iter = 100
step_size = 0.1
cfg, score = hillclimbing(X,y, objective, n_iter, step_size)
print("Cfg = ", cfg)
print("Score = ", score)

Cfg =  [0.5921989964176219, 0.06622761220126494]
Score =  0.8003333333333333


# XGBOOST
+ learning_rate : Tỉ lệ học tập ( 0< lr < 1)
+ n_estimators : Số lượng cây kiểm soát quy mô của quần thể
+ subsample : Tỷ lệ mẫu phụ xác dịnh kích thước mẫu ngẫu nhiên dùng để huấn luyện từng cây ( 0 < subsam < 1)
+ max_depth: độ sâu của cây là số tầng trong mỗi cây( thường từ 1 đến 10)

In [6]:
!pip install xgboost

In [7]:
from xgboost import XGBClassifier

model = XGBClassifier()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

scores = cross_val_score(model, X,y, scoring='accuracy', cv=cv, n_jobs=-1)
print("Score = ", np.mean(scores))
print("Std = ", np.std(scores))


Score =  0.8486666666666667
Std =  0.03989430480100581


In [8]:
def objectives(X,y,cfg):
  lr, n_tree, subsam, depth = cfg
  model = XGBClassifier(learning_rate=lr, n_estimators=3, subsample=subsam, max_depth = depth)
  cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
  scores = cross_val_score(model, X,y, scoring='accuracy', cv=cv, n_jobs=-1)
  result = np.mean(scores)
  return result

def step(cfg):
  lrate, n_tree, subsam, depth = cfg

  lrate = lrate + np.random.randn() * 0.01
  if lrate <= 0.0:
    lrate = 1e-8
  if lrate > 1:
    lrate = 1.0

  n_tree = round(n_tree + np.random.randn() * 50)
  if n_tree <= 0.0:
    n_tree = 1

  subsam = subsam + np.random.randn() * 0.1
  if subsam <= 0.0:
    subsam = 1e-8
  if subsam > 1:
    subsam = 1.0

  depth = round(depth + np.random.randn() * 7)
  if depth <= 1:
    depth = 1
  return [lrate, n_tree, subsam, depth]

def hillclimbing(X, y, objectives, n_iter):
  solution = step([0.1, 100, 1.0, 7])
  solution_eval = objectives(X, y, solution)
  for i in range(n_iter):
    candidate = step(solution)
    candidate_eval = objectives(X, y, candidate)
    if candidate_eval >= solution_eval:
      solution, solution_eval = candidate, candidate_eval
  return [solution, solution_eval]

n_iter = 200
cfg, score = hillclimbing(X, y, objectives, n_iter)
print("cfg = ", (cfg))
print("Score = ", (scores))

cfg =  [0.0761989499826298, 252, 1.0, 4]
Score =  [0.92 0.84 0.86 0.84 0.86 0.87 0.78 0.78 0.89 0.83 0.92 0.81 0.86 0.76
 0.85 0.89 0.82 0.86 0.84 0.89 0.84 0.8  0.87 0.9  0.81 0.84 0.85 0.89
 0.81 0.88]


In [ ]:
print(objectives(X,y,cfg))

0.8753333333333335
